# 实验

In [1]:
from exp import *

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


模型参数量

|model|params|num_layers|hidden_size|num_head|
|---|----|---|----|---|
|gpt2-large|774M|36|1280|20|
|gpt2-xl|1558M|48|1600|25|
|dollm-6B|6064M|48|3200|40|
|dollm-13B|12848M|40|5120|40|

In [2]:
gpt2_large_model = LMConfig(num_layers=36, hidden_size=1280, num_attention_head=20)
gpt2_xl_model = LMConfig(num_layers=48, hidden_size=1600, num_attention_head=25)

对比混合精度训练

In [3]:
compare_exp = {
    "base": Config(gpus_per_node=1, steps=40, batch_size=3, fp16=False, lm=gpt2_large_model),
    "fp16": Config(gpus_per_node=1, steps=40, batch_size=3, fp16=True, lm=gpt2_large_model)
}
compare_metrics = {}

for name, cfg in compare_exp.items():
    train_model(cfg)
    metrics = parse_log("train.log")
    assert metrics is not None
    print_metrics(metrics, name)
    compare_metrics[name] = metrics

training model ......
base: forward-backward: 891.69 ms, optimizer: 65.93 ms,communication: 11.01 ms, compute/total ratio: 0.99, memory: 41,698 M
training model ......
fp16: forward-backward: 363.73 ms, optimizer: 67.55 ms,communication: 5.49 ms, compute/total ratio: 0.99, memory: 28,897 M


In [ ]:
plot_data_dict = {}

for name, metrics in compare_metrics.items():
    metrics: Metrics
    plot_data_dict[name] = {
        "forward-backward": f"{metrics.forward_backward: .2f}",
        "communication": f"{metrics.communication:.2f}",
        "memory": f"{metrics.memory}",
    }

plot_data = pd.DataFrame(plot_data_dict).T
plot(plot_data)